In [1]:
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
# import hvplot.dask
import numpy as np
import dask.array as da
import pickle
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [3]:
# df = pd.read_csv("data/training_set_VU_DM.csv")
df = pd.read_csv("data/train_main.csv")
df.drop('Unnamed: 0', axis = 1, inplace=True)

In [4]:
df

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,27,104.77,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,1,2.20,0.0149,5.03,26,170.74,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,1,2.20,0.0245,4.92,21,179.80,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,1,2.83,0.0125,4.39,34,602.77,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,-1.0,0.0,5.0,-1.0,0.0,5.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,1,2.64,0.1241,4.93,4,143.58,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2974284,332784,2013-06-18 14:35:24,24,219,NaN,NaN,100,114177,4,4.5,1,4.75,0.0909,0.00,15,300.74,0,2142,2,6,1,0,1,0,NaN,328.90,0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,0.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,0.0,19.0,0,NaN,0
2974285,332784,2013-06-18 14:35:24,24,219,NaN,NaN,100,115562,4,4.0,1,4.74,0.0691,0.00,19,213.15,0,2142,2,6,1,0,1,0,NaN,328.83,0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
2974286,332784,2013-06-18 14:35:24,24,219,NaN,NaN,100,121275,4,4.0,0,4.98,0.0385,0.00,25,193.87,1,2142,2,6,1,0,1,0,NaN,329.68,0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2974287,332784,2013-06-18 14:35:24,24,219,NaN,NaN,100,128677,3,3.5,1,4.56,0.1475,0.00,8,107.05,0,2142,2,6,1,0,1,0,NaN,328.72,0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [5]:
original_len = len(df)
original_len

2974289

In [6]:
# 1. Probability if click then book for hotel, if search then click and if search then book
def click_book_df(df):
    grouped_hotels = df.groupby("prop_id").sum(numeric_only=True)
    nr_hotel_found = df["prop_id"].value_counts()
    temp = pd.DataFrame()
    temp["prob_clickbook"] = grouped_hotels["booking_bool"]/ grouped_hotels["click_bool"]
    temp["prob_searchbook"] = grouped_hotels["booking_bool"]/nr_hotel_found
    temp["prob_searchclick"] = grouped_hotels["click_bool"]/nr_hotel_found
    temp.fillna(0,inplace=True)
    temp =temp.reset_index()
    return temp

temp = click_book_df(df)
df = df.merge(temp, on="prop_id")
del temp

In [7]:
df

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,prob_clickbook,prob_searchbook,prob_searchclick
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,27,104.77,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.583333,0.017722,0.03038
1,1495,2013-04-24 02:24:02,5,219,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.98,28,139.00,0,23246,1,40,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.583333,0.017722,0.03038
2,2056,2013-03-07 23:19:22,5,219,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,30,95.00,0,23246,1,1,2,0,1,1,NaN,1244.83,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.583333,0.017722,0.03038
3,2160,2013-06-14 17:09:37,22,92,NaN,NaN,219,893,3,3.5,1,2.83,0.0038,0.00,25,125.82,1,10999,3,94,2,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.583333,0.017722,0.03038
4,2521,2013-04-22 10:54:36,32,220,NaN,NaN,219,893,3,3.5,1,2.83,NaN,4.98,9,126.37,1,5872,2,90,2,0,1,0,NaN,7499.25,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.583333,0.017722,0.03038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2974284,332740,2013-02-18 12:17:59,12,99,NaN,NaN,99,102832,0,2.0,1,2.08,NaN,5.00,20,58.74,0,21461,4,0,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.000000,0.000000,0.00000
2974285,332740,2013-02-18 12:17:59,12,99,NaN,NaN,99,128513,2,3.0,1,1.79,NaN,4.25,8,60.08,0,21461,4,0,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,-1.0,0.0,39.0,NaN,NaN,NaN,0,NaN,0,0.000000,0.000000,0.00000
2974286,332740,2013-02-18 12:17:59,12,99,NaN,NaN,99,129945,0,4.5,1,1.39,NaN,5.00,3,86.78,0,21461,4,0,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,0,0.000000,0.000000,0.00000
2974287,332740,2013-02-18 12:17:59,12,99,NaN,NaN,99,131228,3,3.0,1,0.69,NaN,4.45,2,100.13,0,21461,4,0,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,NaN,0,0.000000,0.000000,0.00000


In [8]:
# 2. Percentile rank of hotel in its country
def hotel_country_rank_df(df):
    grouped_hotels = df.groupby(["prop_id","prop_country_id"]).mean(numeric_only=True).reset_index()
    temp = grouped_hotels[["prop_starrating", "prop_review_score","prop_id","prop_country_id"]].copy()
    temp["total_score"] = temp.loc[:,"prop_starrating"] + temp.loc[:,"prop_review_score"]
    temp["country_pct_rank"] = temp.groupby("prop_country_id")["total_score"].rank(method="dense",
                                                                               ascending=True,pct=True)
    temp = temp[["prop_id","country_pct_rank"]]
    return temp

temp = hotel_country_rank_df(df)
df = df.merge(temp, on="prop_id")
del temp

In [9]:
# 3. Competitors: mean competitor avail/ rate + if there is a better competitor columns.


def competitor_cols(temp:pd.DataFrame, col_names:list,rate_or_avail:str):
    """
    Compute 2 columns for rate and availability. Boolean column if there exists
    a competitor with a better rate and mean competitor values.
    """
    temp_rate = temp[col_names].copy()
    temp_rate_binary = temp_rate.applymap(lambda x: 1 if not pd.isna(x) and x >= 0 else 0)
    temp_rate_binary.loc[:, f"comp_mean_{rate_or_avail}"] = temp_rate.mean(axis=1)
    temp_rate_binary[f"comp_better_{rate_or_avail}"] = temp_rate_binary.any(axis=1).astype(int) #
    temp_rate_binary.drop(columns=col_names,inplace=True)
    return temp_rate_binary

def competitor_df(df):
    inv_cols = [f"comp{i}_inv" for i in np.arange(1,9)]
    rate_cols = [f"comp{i}_rate" for i in np.arange(1,9)]
    rate_perc_cols = [f"comp{i}_rate_percent_diff" for i in np.arange(1,9)]
    
    temp = df.groupby("prop_id").mean(numeric_only=True)
    rate_res = competitor_cols(temp, rate_cols, "rate")
    inv_res = competitor_cols(temp, inv_cols, "inv")
    temp = pd.concat([rate_res, inv_res],axis=1) # only new columns
    return temp

temp = competitor_df(df)
df = df.merge(temp, on="prop_id")# df = 

del temp

# drop original competitor columns
inv_cols = [f"comp{i}_inv" for i in np.arange(1,9)]
rate_cols = [f"comp{i}_rate" for i in np.arange(1,9)]
rate_perc_cols = [f"comp{i}_rate_percent_diff" for i in np.arange(1,9)]

# df.drop(columns=rate_cols+inv_cols+rate_perc_cols, inplace=True)

In [10]:
# 5. Price change in percentile, no NaN so don't need to average
def price_change_col(df):
    old_price = np.exp(df["prop_log_historical_price"])
    return (df["price_usd"] - old_price) / old_price
    
df["price_change"] = price_change_col(df)


In [11]:
# 6. Desirability_score as function of location and affinity score

# Compute the desirability score as a weighted average of the four measures
def desirability_col(df):
    desirability_score = np.exp(df["srch_query_affinity_score"]) *  (df["prop_location_score1"] + df["prop_location_score2"])
    desirability_score.fillna(desirability_score.mean(), inplace=True)
    return desirability_score

df["desirability_score"] = desirability_col(df)

In [12]:
# 7. Date time to percentile of the day

def percent_day_col(df):
    df["date_time"] = pd.to_datetime(df["date_time"])
    return (df['date_time'].dt.hour * 3600 + df['date_time'].dt.minute * 60 + df['date_time'].dt.second) / 86400

def month_col(df):
    df["date_time"] = pd.to_datetime(df["date_time"])
    return df["date_time"].dt.month

def weekend_bool_col(df):
    df["date_time"] = pd.to_datetime(df["date_time"])
    return df["date_time"].dt.day_of_week

df['percentile_of_day'] = percent_day_col(df)
df['month'] = month_col(df)
df['day_of_week'] = weekend_bool_col(df)
# df.drop(columns=["date_time"], inplace=True)

In [13]:
df

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,prob_clickbook,prob_searchbook,prob_searchclick,country_pct_rank,comp_mean_rate,comp_better_rate,comp_mean_inv,comp_better_inv,price_change,desirability_score,percentile_of_day,month,day_of_week
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,27,104.77,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.583333,0.017722,0.03038,0.65,-0.01104,1,-0.003033,1,-0.257871,0.001398,0.355729,4,3
1,1495,2013-04-24 02:24:02,5,219,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.98,28,139.00,0,23246,1,40,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.583333,0.017722,0.03038,0.65,-0.01104,1,-0.003033,1,-0.044505,0.001398,0.100023,4,2
2,2056,2013-03-07 23:19:22,5,219,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,30,95.00,0,23246,1,1,2,0,1,1,NaN,1244.83,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.583333,0.017722,0.03038,0.65,-0.01104,1,-0.003033,1,-0.327076,0.001398,0.971782,3,3
3,2160,2013-06-14 17:09:37,22,92,NaN,NaN,219,893,3,3.5,1,2.83,0.0038,0.00,25,125.82,1,10999,3,94,2,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.583333,0.017722,0.03038,0.65,-0.01104,1,-0.003033,1,124.820000,0.001398,0.715012,6,4
4,2521,2013-04-22 10:54:36,32,220,NaN,NaN,219,893,3,3.5,1,2.83,NaN,4.98,9,126.37,1,5872,2,90,2,0,1,0,NaN,7499.25,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.583333,0.017722,0.03038,0.65,-0.01104,1,-0.003033,1,-0.131325,0.001398,0.454583,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2974284,332740,2013-02-18 12:17:59,12,99,NaN,NaN,99,102832,0,2.0,1,2.08,NaN,5.00,20,58.74,0,21461,4,0,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.000000,0.000000,0.00000,0.20,0.00000,1,0.000000,1,-0.604213,0.001398,0.512488,2,0
2974285,332740,2013-02-18 12:17:59,12,99,NaN,NaN,99,128513,2,3.0,1,1.79,NaN,4.25,8,60.08,0,21461,4,0,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,-1.0,0.0,39.0,NaN,NaN,NaN,0,NaN,0,0.000000,0.000000,0.00000,0.50,-0.50000,1,0.000000,1,-0.143005,0.001398,0.512488,2,0
2974286,332740,2013-02-18 12:17:59,12,99,NaN,NaN,99,129945,0,4.5,1,1.39,NaN,5.00,3,86.78,0,21461,4,0,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,0,0.000000,0.000000,0.00000,0.45,0.00000,1,0.000000,1,-0.415281,0.001398,0.512488,2,0
2974287,332740,2013-02-18 12:17:59,12,99,NaN,NaN,99,131228,3,3.0,1,0.69,NaN,4.45,2,100.13,0,21461,4,0,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,N

In [14]:
def usa_col(df):
    # Encode visitor/hotel from USA (approx. 50% of visitors)
    # idx visitor  hotel  encoding
    # 0     1     1        2
    # 1     0     0        0
    # 2     1     0        1
    # 3     0     1       -1
    visitor_us = (df["visitor_location_country_id"] == 219).astype(int)
    hotel_us = (df["prop_country_id"] == 219).astype(int)
    return np.where(visitor_us != hotel_us, visitor_us - hotel_us, hotel_us*2)

df["within_us"] =  usa_col(df)

In [15]:
# # df.to_csv("data/FE_training_set.csv")
# df.to_csv("data/train_train_FE.csv")

## Test set

- need to add expected click stuff here too, but using values from train set bc. dont have test set values

In [37]:
# df_test = pd.read_csv("data/test_set_VU_DM.csv")
df_test = pd.read_csv("data/train_test.csv")
df_test.drop('Unnamed: 0', axis = 1,  inplace=True)

In [38]:
len_test_original = len(df_test)
len_test_original

496146

In [39]:
df_train = df.copy(deep=True)

#### Copy click, search, booking columns to test dataset
Dataset with len(unique hotels that are in both).

With only the new columns to copy them to the test data for the matching hotels.

In [40]:
test_props = df_test["prop_id"].unique()
train_props = df_train["prop_id"].unique() 

# training dataset, but only the properties that are in the test set too
train_df_intersect = df_train[df_train["prop_id"].isin(np.intersect1d(test_props,train_props))]  

# select the columns we added before
clicks_train = train_df_intersect[["prop_id","prob_clickbook","prob_searchbook",
                                      "prob_searchclick","country_pct_rank"]]

# reduce the nr of rows to unique entries
clicks_train= clicks_train.groupby("prop_id").mean() 
clicks_train["in_train"] = 1 # bool column if was already in training set
clicks_train.reset_index(inplace=True)

In [41]:
df_test = df_test.merge(clicks_train, on="prop_id",how="left") 

In [42]:
df_test["in_train"].fillna(0, inplace=True) # if property not in training set, fill with 0

##### Rank of the hotel must be computed using both the test and training set

#### All other features must be computed for all rows in the test set
- compute hotel rank with whole test set, but only update hotels that are new

In [43]:
# add ranking 
save = df_test.copy()
len(save)

496146

In [44]:
df_test = save.copy()

In [45]:
# apply ranking, filter out new hotels and appy only there
temp = hotel_country_rank_df(df_test)
hotels_not_in_train = df_test[df_test["in_train"]==0]["prop_id"].unique()
temp = temp[temp["prop_id"].isin(hotels_not_in_train)]

# fill nan values of the missing ranks_rank_x","country_pct_rank_y"], inplace=True)

In [46]:
df_test = df_test.merge(temp,on="prop_id",how="left")

In [47]:
#  for the new hotels with the values from the testranking
df_test = df_test.merge(temp,on="prop_id",how="left")
df_test["country_pct_rank"] = df_test['country_pct_rank_x'].combine_first(df_test['country_pct_rank_y'])
df_test.drop(columns=["country_pct_rank_x","country_pct_rank_y"],inplace=True)

In [48]:
df_test = df_test.merge(competitor_df(df_test), on="prop_id")

In [49]:
df_test["price_change"] = price_change_col(df_test)
df_test["desirability_score"] = desirability_col(df_test)
df_test['percentile_of_day'] = percent_day_col(df_test)
df_test['month'] = month_col(df_test)
df_test['day_of_week'] = weekend_bool_col(df_test)
df_test["within_us"] =  usa_col(df_test)

In [50]:
df_test

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,prob_clickbook,prob_searchbook,prob_searchclick,in_train,country_pct_rank,comp_mean_rate,comp_better_rate,comp_mean_inv,comp_better_inv,price_change,desirability_score,percentile_of_day,month,day_of_week,within_us
0,179247,2013-03-24 07:45:31,5,219,NaN,NaN,219,18730,3,4.5,1,1.10,0.0320,4.82,1,100.00,0,4687,2,5,1,1,2,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.4,0.025974,0.064935,1.0,0.75,-0.111111,1,0.175,1,-0.193321,1.549386e-03,0.323275,3,6,2
1,192895,2012-11-04 10:52:01,5,219,NaN,NaN,219,18730,3,4.5,1,1.10,NaN,4.86,21,100.00,0,392,1,0,1,0,1,0,NaN,2.03,0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.4,0.025974,0.064935,1.0,0.75,-0.111111,1,0.175,1,-0.224952,1.549386e-03,0.452789,11,6,2
2,193732,2012-11-06 06:21:36,5,219,2.58,124.29,219,18730,3,4.5,1,1.10,0.0320,4.85,6,115.00,0,4687,2,3,5,0,3,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,1,792.30,1,0.4,0.025974,0.064935,1.0,0.75,-0.111111,1,0.175,1,-0.099737,1.549386e-03,0.265000,11,1,2
3,197644,2012-12-06 15:41:58,5,219,NaN,NaN,219,18730,3,4.5,1,1.10,0.0320,4.84,33,100.00,0,4687,1,18,2,0,1,0,-26.5334,107.92,0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.4,0.025974,0.064935,1.0,0.75,-0.111111,1,0.175,1,-0.209295,3.392636e-12,0.654144,12,3,2
4,199194,2013-03-05 07:32:02,5,219,NaN,NaN,219,18730,3,4.5,1,1.10,NaN,5.03,28,170.00,0,392,1,4,2,0,1,1,NaN,41.22,0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.4,0.025974,0.064935,1.0,0.75,-0.111111,1,0.175,1,0.111598,1.549386e-03,0.313912,3,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496141,262094,2013-02-12 09:45:34,18,129,NaN,NaN,99,121629,4,4.5,0,0.00,NaN,5.77,36,329.61,0,11408,5,3,2,1,1,0,NaN,386.69,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,0,0.0,0.000000,0.000000,1.0,0.85,0.000000,1,0.000,1,0.028303,1.549386e-03,0.406644,2,1,0
496142,262133,2013-04-20 17:56:49,1,13,NaN,NaN,99,10919,2,4.0,1,2.08,NaN,4.46,6,97.71,0,12217,7,104,4,0,2,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,0.0,0.60,NaN,0,NaN,0,0.129759,1.549386e-03,0.747789,4,5,0
496143,262133,2013-04-20 17:56:49,1,13,NaN,NaN,99,77676,2,3.5,1,1.39,NaN,4.46,4,79.34,0,12217,7,104,4,0,2,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,0,0.0,0.000000,0.000000,1.0,0.55,0.000000,1,0.000,1,-0.082642,1.549386e-03,0.747789,4,5,0
496144,262138,2013-01-20 05:47:14,5,219,NaN,NaN,219,7022,0,5.0,1,2.30,0.0328,4.78,34,129.00,0,4352,1,0,2,0,1,0,NaN,1223.00,0,NaN,NaN,NaN,NaN,NaN

In [51]:
# df_test.drop(columns=rate_cols+inv_cols+rate_perc_cols+["date_time"], inplace=True)

In [52]:
len(df_test) - 4959183

-4463037

In [53]:
# both datasets have all the added columns
np.setdiff1d(df_train.columns, df_test.columns)

array([], dtype=object)

In [54]:
# df_test.to_csv("data/FE_test_set.csv")
# df_test.to_csv("data/train_test_FE.csv")

In [55]:
# both datasets have all the added columns
np.setdiff1d(df_train.columns, df_test.columns)

array([], dtype=object)

In [56]:
# df_test.to_csv("data/FE_test_set.csv")
# df_test.to_csv("data/train_test_FE.csv")

In [57]:
pd.concat([df, df_test]).to_csv("data/train_combined_FE.csv")